In [4]:
import pandas as pd
import json
import random

# Read in cvs
abstratcs = 'asreview_dataset_relevant_Psychedelic Study.csv'

df = pd.read_csv(abstratcs)

print(df.columns)
df.rename(columns={'abstract': 'text'}, inplace=True)
relevant_cols = ['record_id',  'keywords', 'title', 'text']
# replace NaN with empty string
df.fillna('', inplace=True)

# Create data json
data = df[relevant_cols].to_dict(orient='records')
nr_of_records = len(data)
random_subset = random.sample(data, 100)
# replace NaN with empty string

output_file = '../prodigy/input/psychdelic_study_100_20240312.jsonl'
# Write first 100 into a jsonl
with open(output_file, 'w', encoding='utf-8') as f:
    for d in random_subset:
        f.write(json.dumps(d) + '\n')

Index(['record_id', 'type_of_reference', 'name_of_database', 'number',
       'keywords', 'language', 'notes', 'year', 'start_page', 'short_title',
       'secondary_title', 'title', 'url', 'volume', 'id', 'type_of_work',
       'issn', 'abstract', 'doi', 'accession_number', 'date',
       'database_provider', 'secondary_authors', 'place_published',
       'publisher', 'tertiary_title', 'edition', 'custom1', 'alternate_title1',
       'author_address', 'authors', 'custom2', 'custom6', 'custom5', 'note',
       'custom7', 'original_publication', 'custom3', 'asreview_prior',
       'exported_notes_1', 'included', 'asreview_ranking'],
      dtype='object')
